In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import pandas as pd
from datetime import timedelta
from datetime import datetime

In [2]:
# define folder and file paths
folder_HMA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"HMA2")
folder_debris = os.path.join(folder_HMA, 'Datasets', 'debris cover')
folder_rgi = os.path.join(folder_HMA, 'Datasets', 'RGI')

# open csvs that allow us to link rgiids between v6 and v7
link_13 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-13_central_asia-rgi6_links.csv'))
link_14 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-14_south_asia_west-rgi6_links.csv'))
link_15 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-15_south_asia_east-rgi6_links.csv'))

In [28]:
# get unique rgi7 ids from link files
id7_unique = link_13['rgi7_id'].unique()
len(id7_unique)

56712

In [29]:
# get the ones that have merged multiple glacier >0.5km2
possible_bads = []
for i in id7_unique:
    # if i != "RGI2000-v7.0-G-15-00060": continue
    
    # subset to this rgiid
    link_subset = link_13[link_13['rgi7_id']==i]
    
    # if it's only linked to a single rgi6 glacier, then assume it's fine
    if len(link_subset)==1: continue
    
    # if it's linked to multiple rgi6 glaciers, check how many are >1 km2
    n_big = len(link_subset[link_subset['overlap_area_km2']>1])
    
    # if there are multiple 'big', then it might be a problem
    if n_big>1:
        possible_bads.append(i)

In [30]:
possible_bads

['RGI2000-v7.0-G-13-08406',
 'RGI2000-v7.0-G-13-14112',
 'RGI2000-v7.0-G-13-14144',
 'RGI2000-v7.0-G-13-14151',
 'RGI2000-v7.0-G-13-14152',
 'RGI2000-v7.0-G-13-14195',
 'RGI2000-v7.0-G-13-14338',
 'RGI2000-v7.0-G-13-14385',
 'RGI2000-v7.0-G-13-14632',
 'RGI2000-v7.0-G-13-16383',
 'RGI2000-v7.0-G-13-16501',
 'RGI2000-v7.0-G-13-16724',
 'RGI2000-v7.0-G-13-16736',
 'RGI2000-v7.0-G-13-17739',
 'RGI2000-v7.0-G-13-18122',
 'RGI2000-v7.0-G-13-18131',
 'RGI2000-v7.0-G-13-18525',
 'RGI2000-v7.0-G-13-18588',
 'RGI2000-v7.0-G-13-20889',
 'RGI2000-v7.0-G-13-23966',
 'RGI2000-v7.0-G-13-24414',
 'RGI2000-v7.0-G-13-24951',
 'RGI2000-v7.0-G-13-24961',
 'RGI2000-v7.0-G-13-26643',
 'RGI2000-v7.0-G-13-27528',
 'RGI2000-v7.0-G-13-27560',
 'RGI2000-v7.0-G-13-27819',
 'RGI2000-v7.0-G-13-28030',
 'RGI2000-v7.0-G-13-28044',
 'RGI2000-v7.0-G-13-28413',
 'RGI2000-v7.0-G-13-28434',
 'RGI2000-v7.0-G-13-28548',
 'RGI2000-v7.0-G-13-29789',
 'RGI2000-v7.0-G-13-29973',
 'RGI2000-v7.0-G-13-30383',
 'RGI2000-v7.0-G-13-

In [31]:
# subset the rgi7 to these glaciers and save
rgi_df = gpd.read_file(os.path.join(folder_rgi, 'v7', 'G', 'RGI2000-v7.0-G-13_central_asia', 'RGI2000-v7.0-G-13_central_asia.shp'))

rgi_save = rgi_df[rgi_df["rgi_id"].isin(possible_bads)]
out_path = os.path.join(folder_rgi, 'possible_bads_13.shp')
rgi_save.to_file(out_path)